In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

In [2]:
links = []
dates = []
for i in range(1,14):
    r1 = requests.get('https://segelyszervezet.hu/sajtokozlemenyek/?offset='+str(i))
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html5lib')
    coverpage_news = soup1.find_all('article',class_="card card--article col-md-3")
    for j in range(len(coverpage_news)):
        date = coverpage_news[j].find('time').get_text()
        link = coverpage_news[j].find('a').get('href','')
        dates.append(date)
        links.append(link)

In [3]:
cikkek_adatai = []
start = time.time()
counter = 0
for m in links:
    # link
    article = requests.get(m)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')

    #date
    date = dates[counter]
    
    # cím
    title_raw = soup_article.select("h1")
    title = title_raw[0].get_text(strip=True)

    # törzsszöveg
    body = soup_article.find_all('p')
    final_article = ""
    for i in range(1, len(body)-12):
        final_article += ' ' + body[i].get_text(strip=True)
    
    # putting it together
    cikkek_adatai.append([date, title, final_article, m])
    end = time.time()
    counter += 1
    print(str(counter) + '/' + str(len(links)), end - start)

1/154 0.26134371757507324
2/154 0.49390482902526855
3/154 0.7888076305389404
4/154 1.0196723937988281
5/154 1.6240556240081787
6/154 1.930804967880249
7/154 2.503053903579712
8/154 3.0630860328674316
9/154 3.682309627532959
10/154 4.2438390254974365
11/154 4.8217384815216064
12/154 5.426513195037842
13/154 6.018049478530884
14/154 6.59523344039917
15/154 7.159686088562012
16/154 7.729557037353516
17/154 8.297425985336304
18/154 8.865810871124268
19/154 9.416821479797363
20/154 10.097960710525513
21/154 10.674547672271729
22/154 11.317127466201782
23/154 11.534371137619019
24/154 12.116861343383789
25/154 12.672349214553833
26/154 13.230222702026367
27/154 13.825726747512817
28/154 14.67838978767395
29/154 15.291759490966797
30/154 15.890515565872192
31/154 16.591797828674316
32/154 17.18891143798828
33/154 17.786601066589355
34/154 18.440212965011597
35/154 19.04986071586609
36/154 19.63731336593628
37/154 20.240993976593018
38/154 20.868761777877808
39/154 21.560397386550903
40/154 22

In [4]:
df = pd.DataFrame(data=cikkek_adatai)
df.columns = ['dátum','cím', 'szöveg', 'link']
with pd.ExcelWriter('okumenikus.xlsx') as writer:  
    df.to_excel(writer, sheet_name='hu')